# Local authority revenue expenditure and financing England: 2019 to 2020 budget individual local authority data


Simple datagrab of respreadsheets into `pandas` dataframes and SQLite database.

Putting the data into a SQLite database gives us a certain amount of flexibility with what we then do with it.

For example, it's trivial to expose a SQL query interface / facet explorer over a SQLite database using `datasette`.

Data from:

2019-20: https://www.gov.uk/government/statistics/local-authority-revenue-expenditure-and-financing-england-2019-to-2020-budget-individual-local-authority-data

2018-19: https://www.gov.uk/government/statistics/local-authority-revenue-expenditure-and-financing-england-2018-to-2019-budget-individual-local-authority-data

In [1]:
import pandas as pd

## Define URLs for the Original Source Datafiles

(We could also download the datafiles and then load them locally.)

In [2]:
url={'rev_ac':{}, 'special_grants':{}}

In [3]:
#Revenue account (RA) budget 2019 to 2020
url['rev_ac'][2020]='https://assets.publishing.service.gov.uk/government/uploads/system/uploads/attachment_data/file/853005/RA_2019-20_data_by_LA_upd.xlsx'
url['rev_ac'][2019]='https://www.gov.uk/government/uploads/system/uploads/attachment_data/file/720343/RA_2018-19_data_by_LA.xlsx'

In [4]:
#Specific and special revenue grants (SG) budget 2019 to 2020
url['special_grants'][2020]='https://assets.publishing.service.gov.uk/government/uploads/system/uploads/attachment_data/file/812518/SG_2019-20_data_by_LA.xlsx'
url['special_grants'][2019]='https://www.gov.uk/government/uploads/system/uploads/attachment_data/file/720344/SG_2018-19_data_by_LA.xlsx'

## Load in the Data

Download the original spreadsheet and load it into a dataframe:

In [5]:
YEAR=2020

In [6]:
xl = pd.ExcelFile(url['rev_ac'][YEAR])

xl.sheet_names

['Front Page', 'LA drop-down', 'RA LA Data 2019-20']

We're going to use various cleaning steps, and whilst it seems that the datafiles are all structured in a similar way, it makes sense to check that they are so they get process in the same way.

We could start to build tests in to check this, but that takes time; so for the small number of files, we can just eyeball the files to make sure they're structured the same way. If we return to this notebook with a bit of spare time, we could perhaps start to work on some meaningful checks/tests that would act as patterns we could apply to other spreadsheets too.

In [7]:
#Parse the required sheet and drop any completely empty columns
df = xl.parse(xl.sheet_names[2]).dropna(how='all', axis=1)
df.head()

,Revenue Account Budget (RA) 2019-20: Revenue Account data,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,...,Unnamed: 252,Unnamed: 253,Unnamed: 254,Unnamed: 255,Unnamed: 256,Unnamed: 257,Unnamed: 258,Unnamed: 259,Unnamed: 260,Unnamed: 261
0,Net current expenditure (£ thousand),NaN,NaN,NaN,NaN,4,5,6,7,8,...,251,252,253,254,255,256,257,258,259,260
1,NaN,NaN,1,NaN,NaN,4,5,6,7,8,...,251,252,253,254,255,256,257,258,259,260
2,NaN,NaN,NaN,NaN,NaN,1,1,1,1,1,...,4,4,4,4,4,4,4,4,5,6
3,NaN,NaN,NaN,NaN,NaN,Education Services,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,HRA surplus / deficit,HRA Reserves,NaN
4,NaN,NaN,NaN,NaN,NaN,110,120,130,140,145,...,4026,4027,4028,4029,4030,4033,4035,4040,4046,4046


In [8]:
#Drop the essentially empty third column
if 'Unnamed: 3' in df.columns:
    df.drop(columns='Unnamed: 3', inplace=True)
    
#Drop the first 3 rows
df = df[3:]

#Add a column for the year
df['Year']=YEAR
#Set the value in the first three (header) rows to Year as well
df.iloc[:3, df.columns.get_loc('Year')]='Year'

#For the first five rows of the first three columns, backfill up the column names
df.iloc[:3,:5] = df.iloc[:3,:5].fillna(method='bfill')

#For the first row, which we'll use as column names, fill across
df.iloc[0] = df.iloc[0].fillna(method='ffill')
df.head()

,Revenue Account Budget (RA) 2019-20: Revenue Account data,Unnamed: 1,Unnamed: 2,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,...,Unnamed: 253,Unnamed: 254,Unnamed: 255,Unnamed: 256,Unnamed: 257,Unnamed: 258,Unnamed: 259,Unnamed: 260,Unnamed: 261,Year
3,E-code,ONS Code,Local authority,Class,Education Services,Education Services,Education Services,Education Services,Education Services,Education Services,...,Housing Revenue Account (HRA) Expenditure,Housing Revenue Account (HRA) Expenditure,Housing Revenue Account (HRA) Expenditure,Housing Revenue Account (HRA) Expenditure,Housing Revenue Account (HRA) Expenditure,Housing Revenue Account (HRA) Expenditure,HRA surplus / deficit,HRA Reserves,HRA Reserves,Year
4,E-code,ONS Code,Local authority,Class,110,120,130,140,145,165,...,4027,4028,4029,4030,4033,4035,4040,4046,4046,Year
5,E-code,ONS Code,Local authority,Class,Early years,Primary schools,Secondary schools,Special schools and alternative provision,Post-16 provision,Other education and community budget,...,Capital expenditure charged to the Housing Rev...,Debt management costs,Transfers to GF,Transfers to MRR,Provision for bad debts (+/-),TOTAL HOUSING REVENUE ACCOUNT (HRA) EXPENDITURE,SURPLUS OR DEFICIT FOR THE YEAR ON HRA SERVICES,Housing Revenue Account (HRA) Reserves at 1 April,Housing Revenue Account (HRA) Reserves at 31 M...,Year
6,E0101,E06000022,Bath & North East Somerset UA,UA,11360,21499,8388,11028,1260,10544,...,0,0,0,0,0,0,0,0,0,2020
7,E0102,E06000023,Bristol UA,UA,31033,173483,121005,32013,9405,19205,...,11617,84,0,25630,2936,122637,0,78614,0,2020


There are rows at the end of the table containing notes, which we should drop from the data table, though we should probably keep a record or note of the somewhere, or at least familiarise outselves with what they say to see if it might have bearing on how we treat the data....

In [9]:
df.tail()

,Revenue Account Budget (RA) 2019-20: Revenue Account data,Unnamed: 1,Unnamed: 2,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,...,Unnamed: 253,Unnamed: 254,Unnamed: 255,Unnamed: 256,Unnamed: 257,Unnamed: 258,Unnamed: 259,Unnamed: 260,Unnamed: 261,Year
459,R Amended LCTS data for 4 shire district auth...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2020
460,These data are consistent with the data used t...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2020
461,https://www.gov.uk/government/collections/loca...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2020
462,"Source: Ministry of Housing,Communities and Lo...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2020
463,"Produced on a non-IAS19 and PFI ""Off Balance S...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2020


In [10]:
#Set the column names to the first row values
df.columns = df.iloc[0]

#Drop rows where the "ONS Code" is missing (eg notes rows at end)
df = df.dropna(subset=['ONS Code'])

#Set row index values
df = df.set_index(['Year', 'E-code', 'ONS Code', 'Local authority', 'Class'])
df.tail()

3                                                                              Education Services  \
Year E-code ONS Code  Local authority                                    Class                      
2020 E7051  E23000035 Devon & Cornwall Police and Crime Commissioner ... O                      0   
     E7052  E23000030 Hampshire Police and Crime Commissioner and Chi... O                      0   
     E7053  E23000033 Sussex Police and Crime Commissioner and Chief ... O                      0   
     E7054  E23000029 Thames Valley Police and Crime Commissioner and... O                      0   
     E7055  E23000016 West Mercia Police and Crime Commissioner and C... O                      0   

3                                                                              Education Services  \
Year E-code ONS Code  Local authority                                    Class                      
2020 E7051  E23000035 Devon & Cornwall Police and Crime Commissioner ... O                      0   
     E7052  E23000030 Hampshire Police and Crime Commissioner and Chi... O                      0   
     E7053  E23000033 Sussex Police and Crime Commissioner and Chief ... O                      0   
     E7054  E23000029 Thames Valley Police and Crime Commissioner and... O                      0   
     E7055  E23000016 West Mercia Police and Crime Commissioner and C... O                      0   

3                                                                              Education Services  \
Year E-code ONS Code  Local authority                                    Class                      
2020 E7051  E23000035 Devon & Cornwall Police and Crime Commissioner ... O                      0   
     E7052  E23000030 Hampshire Police and Crime Commissioner and Chi... O                      0   
     E7053  E23000033 Sussex Police and Crime Commissioner and Chief ... O                      0   
     E7054  E23000029 Thames Valley Police and Crime Commissioner and... O                      0   
     E7055  E23000016 West Mercia Police and Crime Commissioner and C... O                      0   

3                                                                              Education Services  \
Year E-code ONS Code  Local authority                                    Class                      
2020 E7051  E23000035 Devon & Cornwall Police and Crime Commissioner ... O                      0   
     E7052  E23000030 Hampshire Police and Crime Commissioner and Chi... O                      0   
     E7053  E23000033 Sussex Police and Crime Commissioner and Chief ... O                      0   
     E7054  E23000029 Thames Valley Police and Crime Commissioner and... O                      0   
     E7055  E23000016 West Mercia Police and Crime Commissioner and C... O                      0   

3                                                                              Education Services  \
Year E-code ONS Code  Local authority                                    Class                      
2020 E7051  E23000035 Devon & Cornwall Police and Crime Commissioner ... O                      0   
     E7052  E23000030 Hampshire Police and Crime Commissioner and Chi... O                      0   
     E7053  E23000033 Sussex Police and Crime Commissioner and Chief ... O                      0   
     E7054  E23000029 Thames Valley Police and Crime Commissioner and... O                      0   
     E7055  E23000016 West Mercia Police and Crime Commissioner and C... O                      0   

3                                                                              Education Services  \
Year E-code ONS Code  Local authority                                    Class                      
2020 E7051  E23000035 Devon & Cornwall Police and Crime Commissioner ... O                      0   
     E7052  E23000030 Hampshire Police and Crime Commissioner and Chi... O                      0   
     E7053  E23000033 Sussex Police and Crime Commission

In [11]:
#Create multi-index column headings from the top 3 rows
cols = list(zip(df.iloc[0,:], df.iloc[1,:], df.iloc[2,:]))
df.columns = pd.MultiIndex.from_tuples(cols)

#Drop the top 3 rows that were used for column headings
df = df.iloc[3:]
df.head()

Education Services  \
                                                                        110    
                                                                 Early years   
Year E-code ONS Code  Local authority               Class                      
2020 E0101  E06000022 Bath & North East Somerset UA UA                 11360   
     E0102  E06000023 Bristol UA                    UA                 31033   
     E0103  E06000025 South Gloucestershire UA      UA                 16780   
     E0104  E06000024 North Somerset UA             UA                 10496   
     E0201  E06000032 Luton UA                      UA                 19829   

                                                                           \
                                                                     120    
                                                          Primary schools   
Year E-code ONS Code  Local authority               Class                   
2020 E0101  E06000022 Bath & North East Somerset UA UA              21499   
     E0102  E06000023 Bristol UA                    UA             173483   
     E0103  E06000025 South Gloucestershire UA      UA              90881   
     E0104  E06000024 North Somerset UA             UA              22707   
     E0201  E06000032 Luton UA                      UA              91222   

                                                                             \
                                                                       130    
                                                          Secondary schools   
Year E-code ONS Code  Local authority               Class                     
2020 E0101  E06000022 Bath & North East Somerset UA UA                 8388   
     E0102  E06000023 Bristol UA                    UA               121005   
     E0103  E06000025 South Gloucestershire UA      UA                10897   
     E0104  E06000024 North Somerset UA             UA                19652   
     E0201  E06000032 Luton UA                      UA                36469   

                                                                                                     \
                                                                                               140    
                                                          Special schools and alternative provision   
Year E-code ONS Code  Local authority               Class                                             
2020 E0101  E06000022 Bath & North East Somerset UA UA                                        11028   
     E0102  E06000023 Bristol UA                    UA                                        32013   
     E0103  E06000025 South Gloucestershire UA      UA                                        16678   
     E0104  E06000024 North Somerset UA             UA                                         1084   
     E0201  E06000032 Luton UA                      UA                                        15417   

                                                                             \
                                                                       145    
                                                          Post-16 provision   
Year E-code ONS Code  Local authority               Class                     
2020 E0101  E06000022 Bath & North East Somerset UA UA                 1260   
     E0102  E06000023 Bristol UA                    UA                 9405   
     E0103  E06000025 South Gloucestershire UA      UA                 3690   
     E0104  E06000024 North Somerset UA             UA                    0   
     E0201  E06000032 Luton UA                      UA                 2620   

                                                                                                \
                                                                                          165    
                                                          Other education and community budget 

The table as it stands can be spilt into several smaller tables, based on the top level column heading:

In [12]:
di = {}

#For each of the top level headings
for i in df.columns.levels[0]:
    #Create a separate subtable
    di[i] = df[i]
    
    #Take the precaution of dropping any empty columns and rows
    di[i] = di[i].dropna(how='all', axis=0).dropna(how='all', axis=1)

#Display subtable names
di.keys()

dict_keys(['Adult Social Care', 'Adult Social Care Precept', 'Capital Items', 'Central Services', "Children's Social Care", 'Cultural and Related Services', 'Education Services', 'Environmental and Regulatory Services', 'Estimated Reserves at 1 April 2019', 'Estimated Reserves at 31 March 2020', 'Fire', 'HRA Reserves', 'HRA surplus / deficit', 'Highways and Transport', 'Housing Revenue Account (HRA) Expenditure', 'Housing Revenue Account (HRA) Income', 'Housing Services (General Fund Revenue Account [GFRA] only)', 'Housing benefits', 'Local Council Tax Support (LCTS)', 'Net Current Expenditure', 'Non-current Expenditure and Receipts', 'Optional detail on protected reserves within Line 1015', 'Other', 'Planning and Development Services', 'Police', 'Precepts and Levies', 'Public Health', 'Revenue Expenditure', 'Revenue Expenditure Financing', 'Total Service\nExpenditure', 'Trading Accounts and Adjustments'])

The column names are pretty horrible to query over, so it can make thimgs easier if we cast the data table in to a long form.

In [13]:
di['Adult Social Care'].reset_index().melt(id_vars=['Year', 'E-code',
                                                    'ONS Code','Local authority',
                                                    'Class'],
                                          var_name=['CodeNum','Section'])

,Year,E-code,ONS Code,Local authority,Class,CodeNum,Section,value
0,2020,E0101,E06000022,Bath & North East Somerset UA,UA,332,Physical support - adults (18–64),1930
1,2020,E0102,E06000023,Bristol UA,UA,332,Physical support - adults (18–64),17999
2,2020,E0103,E06000025,South Gloucestershire UA,UA,332,Physical support - adults (18–64),7444
3,2020,E0104,E06000024,North Somerset UA,UA,332,Physical support - adults (18–64),9311
4,2020,E0201,E06000032,Luton UA,UA,332,Physical support - adults (18–64),7479
...,...,...,...,...,...,...,...,...
8260,2020,E7051,E23000035,Devon & Cornwall Police and Crime Commissioner...,O,360,TOTAL ADULT SOCIAL CARE,0
8261,2020,E7052,E23000030,Hampshire Police and Crime Commissioner and Ch...,O,360,TOTAL ADULT SOCIAL CARE,0
8262,2020,E7053,E23000033,Sussex Police and Crime Commissioner and Chief...,O,360,TOTAL ADULT SOCIAL CARE,0
8263,2020,E7054,E23000029,Thames Valley Police and Crime Commissioner an...,O,360,TOTAL ADULT SOCIAL CARE,0


If required, we can also get a version of the wide table with a simpler, single level set of  column headings.

In [14]:
#Make a copy of the data in a particular area of the full datatable
df = di['Adult Social Care'][:]
#Reset the multi-index column headings to simple column headings
df.T.reset_index(level=[0], drop=True).T.head()

Physical support - adults (18–64)  \
Year E-code ONS Code  Local authority               Class                                     
2020 E0101  E06000022 Bath & North East Somerset UA UA                                 1930   
     E0102  E06000023 Bristol UA                    UA                                17999   
     E0103  E06000025 South Gloucestershire UA      UA                                 7444   
     E0104  E06000024 North Somerset UA             UA                                 9311   
     E0201  E06000032 Luton UA                      UA                                 7479   

                                                          Physical support - older people (65+)  \
Year E-code ONS Code  Local authority               Class                                         
2020 E0101  E06000022 Bath & North East Somerset UA UA                                    11838   
     E0102  E06000023 Bristol UA                    UA                                    40940   
     E0103  E06000025 South Gloucestershire UA      UA                                    14225   
     E0104  E06000024 North Somerset UA             UA                                    13519   
     E0201  E06000032 Luton UA                      UA                                    12178   

                                                          Sensory support - adults (18–64)  \
Year E-code ONS Code  Local authority               Class                                    
2020 E0101  E06000022 Bath & North East Somerset UA UA                                2155   
     E0102  E06000023 Bristol UA                    UA                                1258   
     E0103  E06000025 South Gloucestershire UA      UA                                 286   
     E0104  E06000024 North Somerset UA             UA                                   6   
     E0201  E06000032 Luton UA                      UA                                 106   

                                                          Sensory support - older people (65+)  \
Year E-code ONS Code  Local authority               Class                                        
2020 E0101  E06000022 Bath & North East Somerset UA UA                                     556   
     E0102  E06000023 Bristol UA                    UA                                    1401   
     E0103  E06000025 South Gloucestershire UA      UA                                     265   
     E0104  E06000024 North Somerset UA             UA                                       7   
     E0201  E06000032 Luton UA                      UA                                      95   

                                                          Support with memory and cognition - adults (18–64)  \
Year E-code ONS Code  Local authority               Class                                                      
2020 E0101  E06000022 Bath & North East Somerset UA UA                                                   195   
     E0102  E06000023 Bristol UA                    UA                                                  1094   
     E0103  E06000025 South Gloucestershire UA      UA                                                    36   
     E0104  E06000024 North Somerset UA             UA                                                   354   
     E0201  E06000032 Luton UA                      UA                                                   572   

                                                          Support with memory and cognition - older people (65+)  \
Year E-code ONS Code  Local authority               Class                                                          
2020 E0101  E06000022 Bath & North East Somerset UA UA                                                  8035       
     E0102  E06000023 Bristol UA                    UA                                                 10849       
     E0103  E06000025 South Gloucestershire UA      UA                                                 

In [15]:
def get_spreadsheet_datafile(typ,year, display_names=True):
    """Download spreadsheet from URL and open it as as collection of dataframes."""
    
    _url =  pd.ExcelFile(url[typ][YEAR])
    xl = pd.ExcelFile(_url)
    
    if display_names:
        print('Sheetnames are:\n')
        for i, s in enumerate(xl.sheet_names):
            print(f'\t[{i}] {s}')

    return xl

In [16]:
def get_sheet(xl, sheet_name, year):
    """Extract and parse sheet from a collection of dataframes."""

    _df = xl.parse(sheet_name).dropna(how='all', axis=1)
    
    #Drop the essentially empty third column
    if 'Unnamed: 3' in _df.columns:
        _df.drop(columns='Unnamed: 3', inplace=True)
    
    #Drop the first 3 rows
    _df = _df[3:]
    
    #Add a column for the year
    _df['Year']=YEAR
    #Set the value in the first three (header) rows to Year as well
    _df.iloc[:3, _df.columns.get_loc('Year')]='Year'

    #For the first five rows of the first three columns, backfill up the column names
    _df.iloc[0:3,:5] = _df.iloc[0:3,:5].fillna(method='bfill')
    
    #For the first row, which we'll use as column names, fill across
    _df.iloc[0] = _df.iloc[0].fillna(method='ffill')
    
    #Set the column names to the first row values
    _df.columns = _df.iloc[0]

    #Drop rows where the "ONS Code" is missing (eg notes rows at end)
    _df = _df.dropna(subset=['ONS Code'])

    #Set row index values
    _df = _df.set_index(['Year', 'E-code', 'ONS Code', 'Local authority', 'Class'])

    #Create multi-index column headings from the top 3 rows
    cols = list(zip(_df.iloc[0,:], _df.iloc[1,:], _df.iloc[2,:]))
    _df.columns = pd.MultiIndex.from_tuples(cols)

    #Drop the top 3 rows that were used for column headings
    _df = _df.iloc[3:]

    return _df

In [17]:
def get_subtables_wide(_df):
    """Generate wide subtables from multi-index dataframe."""
    _di = {}

    #For each of the top level headings
    for i in _df.columns.levels[0]:
        #Create a separate subtable
        # - make a copy of the data in a particular area of the full datatable
        _di[i] = _df[i][:]
        # - reset the multi-index column headings to simple column headings
        _di[i] = _df[i].T.reset_index(level=[0], drop=True).T
    
        #Take the precaution of dropping any empty columns and rows
        _di[i] = _di[i].dropna(how='all', axis=0).dropna(how='all', axis=1)

    return _di

In [18]:
def get_subtables_long(_df):
    """Generate long subtables from multi-index dataframe."""
    _di = {}

    #For each of the top level headings
    for i in _df.columns.levels[0]:
        #Create a separate subtable
        # - make a copy of the data in a particular area of the full datatable
        _di[i] = _df[i][:]
 
    #Melt each dataframe from wide to long format
    for t in _di:
        _di[t] = _di[t].reset_index().melt(id_vars=['Year', 'E-code',
                                                    'ONS Code','Local authority',
                                                    'Class'],
                                           var_name=['CodeNum','Section'])
    return _di

In [19]:
year = 2020
typ = 'rev_ac'

xl = get_spreadsheet_datafile(typ,year)

Sheetnames are:

	[0] Front Page
	[1] LA drop-down
	[2] RA LA Data 2019-20


In [20]:
df = get_sheet(xl, xl.sheet_names[2], year)
df.head()

Education Services  \
                                                                        110    
                                                                 Early years   
Year E-code ONS Code  Local authority               Class                      
2020 E0101  E06000022 Bath & North East Somerset UA UA                 11360   
     E0102  E06000023 Bristol UA                    UA                 31033   
     E0103  E06000025 South Gloucestershire UA      UA                 16780   
     E0104  E06000024 North Somerset UA             UA                 10496   
     E0201  E06000032 Luton UA                      UA                 19829   

                                                                           \
                                                                     120    
                                                          Primary schools   
Year E-code ONS Code  Local authority               Class                   
2020 E0101  E06000022 Bath & North East Somerset UA UA              21499   
     E0102  E06000023 Bristol UA                    UA             173483   
     E0103  E06000025 South Gloucestershire UA      UA              90881   
     E0104  E06000024 North Somerset UA             UA              22707   
     E0201  E06000032 Luton UA                      UA              91222   

                                                                             \
                                                                       130    
                                                          Secondary schools   
Year E-code ONS Code  Local authority               Class                     
2020 E0101  E06000022 Bath & North East Somerset UA UA                 8388   
     E0102  E06000023 Bristol UA                    UA               121005   
     E0103  E06000025 South Gloucestershire UA      UA                10897   
     E0104  E06000024 North Somerset UA             UA                19652   
     E0201  E06000032 Luton UA                      UA                36469   

                                                                                                     \
                                                                                               140    
                                                          Special schools and alternative provision   
Year E-code ONS Code  Local authority               Class                                             
2020 E0101  E06000022 Bath & North East Somerset UA UA                                        11028   
     E0102  E06000023 Bristol UA                    UA                                        32013   
     E0103  E06000025 South Gloucestershire UA      UA                                        16678   
     E0104  E06000024 North Somerset UA             UA                                         1084   
     E0201  E06000032 Luton UA                      UA                                        15417   

                                                                             \
                                                                       145    
                                                          Post-16 provision   
Year E-code ONS Code  Local authority               Class                     
2020 E0101  E06000022 Bath & North East Somerset UA UA                 1260   
     E0102  E06000023 Bristol UA                    UA                 9405   
     E0103  E06000025 South Gloucestershire UA      UA                 3690   
     E0104  E06000024 North Somerset UA             UA                    0   
     E0201  E06000032 Luton UA                      UA                 2620   

                                                                                                \
                                                                                          165    
                                                          Other education and community budget 

In [21]:
di = get_subtables_wide(df)
di.keys()

dict_keys(['Adult Social Care', 'Adult Social Care Precept', 'Capital Items', 'Central Services', "Children's Social Care", 'Cultural and Related Services', 'Education Services', 'Environmental and Regulatory Services', 'Estimated Reserves at 1 April 2019', 'Estimated Reserves at 31 March 2020', 'Fire', 'HRA Reserves', 'HRA surplus / deficit', 'Highways and Transport', 'Housing Revenue Account (HRA) Expenditure', 'Housing Revenue Account (HRA) Income', 'Housing Services (General Fund Revenue Account [GFRA] only)', 'Housing benefits', 'Local Council Tax Support (LCTS)', 'Net Current Expenditure', 'Non-current Expenditure and Receipts', 'Optional detail on protected reserves within Line 1015', 'Other', 'Planning and Development Services', 'Police', 'Precepts and Levies', 'Public Health', 'Revenue Expenditure', 'Revenue Expenditure Financing', 'Total Service\nExpenditure', 'Trading Accounts and Adjustments'])

In [22]:
di['Adult Social Care'].head(3)

Physical support - adults (18–64)  \
Year E-code ONS Code  Local authority               Class                                     
2020 E0101  E06000022 Bath & North East Somerset UA UA                                 1930   
     E0102  E06000023 Bristol UA                    UA                                17999   
     E0103  E06000025 South Gloucestershire UA      UA                                 7444   

                                                          Physical support - older people (65+)  \
Year E-code ONS Code  Local authority               Class                                         
2020 E0101  E06000022 Bath & North East Somerset UA UA                                    11838   
     E0102  E06000023 Bristol UA                    UA                                    40940   
     E0103  E06000025 South Gloucestershire UA      UA                                    14225   

                                                          Sensory support - adults (18–64)  \
Year E-code ONS Code  Local authority               Class                                    
2020 E0101  E06000022 Bath & North East Somerset UA UA                                2155   
     E0102  E06000023 Bristol UA                    UA                                1258   
     E0103  E06000025 South Gloucestershire UA      UA                                 286   

                                                          Sensory support - older people (65+)  \
Year E-code ONS Code  Local authority               Class                                        
2020 E0101  E06000022 Bath & North East Somerset UA UA                                     556   
     E0102  E06000023 Bristol UA                    UA                                    1401   
     E0103  E06000025 South Gloucestershire UA      UA                                     265   

                                                          Support with memory and cognition - adults (18–64)  \
Year E-code ONS Code  Local authority               Class                                                      
2020 E0101  E06000022 Bath & North East Somerset UA UA                                                   195   
     E0102  E06000023 Bristol UA                    UA                                                  1094   
     E0103  E06000025 South Gloucestershire UA      UA                                                    36   

                                                          Support with memory and cognition - older people (65+)  \
Year E-code ONS Code  Local authority               Class                                                          
2020 E0101  E06000022 Bath & North East Somerset UA UA                                                  8035       
     E0102  E06000023 Bristol UA                    UA                                                 10849       
     E0103  E06000025 South Gloucestershire UA      UA                                                  7038       

                                                          Learning disability support - adults (18–64)  \
Year E-code ONS Code  Local authority               Class                                                
2020 E0101  E06000022 Bath & North East Somerset UA UA                                           14890   
     E0102  E06000023 Bristol UA                    UA                                           48450   
     E0103  E06000025 South Gloucestershire UA      UA                                           38147   

                                                          Learning disability support - older people (65+)  \
Year E-code ONS Code  Local authority               Class                                                    
2020 E0101  E06000022 Bath & North East Somerset UA UA                                                1829   
     E0102  E06000023 Bristol UA                    UA                                                7619   
     E0103 

We can also get this data as a list of Python `dict`s:

In [23]:
di['Adult Social Care'].head(2).reset_index().to_dict(orient='records')

[{'Year': 2020,
  'E-code': 'E0101',
  'ONS Code': 'E06000022',
  'Local authority': 'Bath & North East Somerset UA',
  'Class': 'UA',
  'Physical support - adults (18–64)': 1930,
  'Physical support - older people (65+)': 11838,
  'Sensory support - adults (18–64)': 2155,
  'Sensory support - older people (65+)': 556,
  'Support with memory and cognition - adults (18–64)': 195,
  'Support with memory and cognition - older people (65+)': 8035,
  'Learning disability support - adults (18–64)': 14890,
  'Learning disability support - older people (65+)': 1829,
  'Mental health support - adults (18–64)': 3636,
  'Mental health support - older people (65+)': 6666,
  'Social support: Substance misuse support': 505,
  'Social support: Asylum seeker support': 0,
  'Social support: Support for carer': 38,
  'Social support: Social Isolation': 867,
  'Assistive equipment and technology': 275,
  'Social care activities': 3657,
  'Information and early intervention': 1503,
  'Commissioning and se

In [24]:
year = 2020
typ = 'special_grants'

xl = get_spreadsheet_datafile(typ, year)

Sheetnames are:

	[0] Front Page
	[1] LA drop-down
	[2] SG LA Data 2019-20


In [25]:
get_sheet(xl, xl.sheet_names[2], year).head()

Specific And Special Grants Within Aggregate External Finance (AEF)  \
                                                                                                                          102   
                                                                                               Dedicated Schools Grant (DSG)    
Year E-code ONS Code  Local authority               Class                                                                       
2020 E0101  E06000022 Bath & North East Somerset UA UA                                                 53701                    
     E0102  E06000023 Bristol UA                    UA                                                356911                    
     E0103  E06000025 South Gloucestershire UA      UA                                                133886                    
     E0104  E06000024 North Somerset UA             UA                                                 47555                    
     E0201  E06000032 Luton UA                      UA                                                154796                    

                                                                               \
                                                                          103   
                                                          Pupil Premium Grant   
Year E-code ONS Code  Local authority               Class                       
2020 E0101  E06000022 Bath & North East Somerset UA UA                      0   
     E0102  E06000023 Bristol UA                    UA                   7229   
     E0103  E06000025 South Gloucestershire UA      UA                   4810   
     E0104  E06000024 North Somerset UA             UA                   2276   
     E0201  E06000032 Luton UA                      UA                   9490   

                                                                                               \
                                                                                          107   
                                                          Universal Infants Free School Meals   
Year E-code ONS Code  Local authority               Class                                       
2020 E0101  E06000022 Bath & North East Somerset UA UA                                      0   
     E0102  E06000023 Bristol UA                    UA                                   2787   
     E0103  E06000025 South Gloucestershire UA      UA                                   2918   
     E0104  E06000024 North Somerset UA             UA                                   2211   
     E0201  E06000032 Luton UA                      UA                                      0   

                                                                                   \
                                                                              221   
                                                          GLA Transport Grant (b)   
Year E-code ONS Code  Local authority               Class                           
2020 E0101  E06000022 Bath & North East Somerset UA UA                          0   
     E0102  E06000023 Bristol UA                    UA                          0   
     E0103  E06000025 South Gloucestershire UA      UA                          0   
     E0104  E06000024 North Somerset UA             UA                          0   
     E0201  E06000032 Luton UA                      UA                          0   

                                                                                \
                                                                           313   
                                                          Public Health Grant    
Year E-code ONS Code  Local authority               Class                        
2020 E0101  E06000022 Bath & North East Somerset UA UA                    8694   
     E0102  E06000023 Bristol UA                    UA                   31628   
     E0103  E06000025 South Glouce

## Save Data Tables To SQLite

To persist the data, we can store it in a SQLite database.

This may seem like extra effort, but if we keep adding tables and datasets to the same database, we can start to run queries that pull in data from multiple tables, which often leads to new insights.

We can also start to build tools around particular tables, or column sets. For example, tools to map datasets using things like choropleth maps.

*[TO DO; see https://github.com/ouseful-demos/choropleth-map-demo which we could perhaps build on to do an ONS code based mapper? Maybe even as a datasette plugin?]*

In [26]:
import sqlite_utils

In [27]:
di.keys()

dict_keys(['Adult Social Care', 'Adult Social Care Precept', 'Capital Items', 'Central Services', "Children's Social Care", 'Cultural and Related Services', 'Education Services', 'Environmental and Regulatory Services', 'Estimated Reserves at 1 April 2019', 'Estimated Reserves at 31 March 2020', 'Fire', 'HRA Reserves', 'HRA surplus / deficit', 'Highways and Transport', 'Housing Revenue Account (HRA) Expenditure', 'Housing Revenue Account (HRA) Income', 'Housing Services (General Fund Revenue Account [GFRA] only)', 'Housing benefits', 'Local Council Tax Support (LCTS)', 'Net Current Expenditure', 'Non-current Expenditure and Receipts', 'Optional detail on protected reserves within Line 1015', 'Other', 'Planning and Development Services', 'Police', 'Precepts and Levies', 'Public Health', 'Revenue Expenditure', 'Revenue Expenditure Financing', 'Total Service\nExpenditure', 'Trading Accounts and Adjustments'])

In [28]:
#!rm la_revenue.db
db = sqlite_utils.Database("la_revenue.db")

In [29]:
def df_as_dictlist(_df):
    """Return a list of dicts corresponding to a dataframe."""
    return _df.to_dict(orient='records')

In [30]:
def clean_table_names(n):
    """Create clean / easily queried table names."""
    
    def _clean_char(s,c,w=''):
        """Clean a list characters from a string."""
        for _c in c:
            s = s.replace(_c,w)
        return s
    
    clean_names = {}
    for _n in n:
        clean_name = _clean_char(_n.lower(), ['\n','\r'], ' ')
        clean_name =  _clean_char(clean_name, ['(',')','[', ']', '\\','/',"'",'"'] )
        clean_names[_n] = clean_name.replace(' ','_')
        
    return clean_names

In [31]:
clean_table_names(di.keys())

{'Adult Social Care': 'adult_social_care',
 'Adult Social Care Precept': 'adult_social_care_precept',
 'Capital Items': 'capital_items',
 'Central Services': 'central_services',
 "Children's Social Care": 'childrens_social_care',
 'Cultural and Related Services': 'cultural_and_related_services',
 'Education Services': 'education_services',
 'Environmental and Regulatory Services': 'environmental_and_regulatory_services',
 'Estimated Reserves at 1 April 2019': 'estimated_reserves_at_1_april_2019',
 'Estimated Reserves at 31 March 2020': 'estimated_reserves_at_31_march_2020',
 'Fire': 'fire',
 'HRA Reserves': 'hra_reserves',
 'HRA surplus / deficit': 'hra_surplus__deficit',
 'Highways and Transport': 'highways_and_transport',
 'Housing Revenue Account (HRA) Expenditure': 'housing_revenue_account_hra_expenditure',
 'Housing Revenue Account (HRA) Income': 'housing_revenue_account_hra_income',
 'Housing Services (General Fund Revenue Account [GFRA] only)': 'housing_services_general_fund_rev

In [32]:
#for each spreadsheet
for typ in url:
    #for each year
    for year in url[typ]:
        #Get the spreadsheet
        xl = get_spreadsheet_datafile(typ,year, display_names=False)
        #Get the sheet
        df = get_sheet(xl, xl.sheet_names[2], year)
        #Get the long form subtables
        di = get_subtables_long(df)
        #Generate clean table names from each subtable name
        cn = clean_table_names(di.keys())
        #For each subtable
        for t in di.keys():
            #Add it to the database
            db[cn[t]].insert_all(df_as_dictlist(di[t]))
            

/usr/local/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3051: PerformanceWarning: indexing past lexsort depth may impact performance.
  interactivity=interactivity, compiler=compiler, result=result)
/usr/local/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3319: PerformanceWarning: indexing past lexsort depth may impact performance.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [33]:
db.table_names()

['adult_social_care',
 'adult_social_care_precept',
 'capital_items',
 'central_services',
 'childrens_social_care',
 'cultural_and_related_services',
 'education_services',
 'environmental_and_regulatory_services',
 'estimated_reserves_at_1_april_2019',
 'estimated_reserves_at_31_march_2020',
 'fire',
 'hra_reserves',
 'hra_surplus__deficit',
 'highways_and_transport',
 'housing_revenue_account_hra_expenditure',
 'housing_revenue_account_hra_income',
 'housing_services_general_fund_revenue_account_gfra_only',
 'housing_benefits',
 'local_council_tax_support_lcts',
 'net_current_expenditure',
 'non-current_expenditure_and_receipts',
 'optional_detail_on_protected_reserves_within_line_1015',
 'other',
 'planning_and_development_services',
 'police',
 'precepts_and_levies',
 'public_health',
 'revenue_expenditure',
 'revenue_expenditure_financing',
 'total_service_expenditure',
 'trading_accounts_and_adjustments',
 'specific_and_special_grants_outside_aggregate_external_finance_aef',
 's

## Query the Database

We can query the database quite straightforwardly using `sqlite_utils` methods or SQL queries.

In [34]:
db["adult_social_care"].get(1)

{'Year': 2020,
 'E-code': 'E0101',
 'ONS Code': 'E06000022',
 'Local authority': 'Bath & North East Somerset UA',
 'Class': 'UA',
 'CodeNum': 332,
 'Section': 'Physical support - adults (18–64)',
 'value': 1930}

In [35]:
[r for r in db["adult_social_care"].rows_where('`ONS Code`="E06000025" LIMIT 2')]

[{'Year': 2020,
  'E-code': 'E0103',
  'ONS Code': 'E06000025',
  'Local authority': 'South Gloucestershire UA',
  'Class': 'UA',
  'CodeNum': 332,
  'Section': 'Physical support - adults (18–64)',
  'value': 7444},
 {'Year': 2020,
  'E-code': 'E0103',
  'ONS Code': 'E06000025',
  'Local authority': 'South Gloucestershire UA',
  'Class': 'UA',
  'CodeNum': 333,
  'Section': 'Physical support - older people (65+)',
  'value': 14225}]

In [36]:
pd.read_sql("SELECT * FROM adult_social_care LIMIT 3", db.conn)

,Year,E-code,ONS Code,Local authority,Class,CodeNum,Section,value
0,2020,E0101,E06000022,Bath & North East Somerset UA,UA,332,Physical support - adults (18–64),1930
1,2020,E0102,E06000023,Bristol UA,UA,332,Physical support - adults (18–64),17999
2,2020,E0103,E06000025,South Gloucestershire UA,UA,332,Physical support - adults (18–64),7444


It might make sense to standardise on some rather more easily queried column names.

This needs to be handled further up the notebook.

For example:

- `E-code`: `eCode`;
- `ONS Code`: `ONScode`;
- `Local authority`: `LA`

